In [4]:
import pandas as pd
from sqlalchemy import create_engine


dbname = 'DataMining'
user = 'postgres'
password = 'datamining'
host = 'localhost'  # localhost or the server address
port = '5433'  # default PostgreSQL port is 5432

# Establish a connection to the database
connection_str = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(connection_str)

In [10]:
query = """
SELECT 
    timestamps_utc, 
    lat, 
    lon, 
    rs_e_inairtemp_pc1, 
    rs_e_inairtemp_pc2, 
    rs_e_oilpress_pc1, 
    rs_e_oilpress_pc2, 
    rs_e_rpm_pc1, 
    rs_e_rpm_pc2, 
    rs_e_wattemp_pc1, 
    rs_e_wattemp_pc2, 
    rs_t_oiltemp_pc1, 
    rs_t_oiltemp_pc2
FROM 
    vehicle_data;
"""


# Execute the query and fetch the data into a DataFrame
df = pd.read_sql_query(query, engine)

# Close the database connection
engine.dispose()

NORMALIZE DATA FOR PROXIMITY BASED CLUSTERING

In [13]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import LocalOutlierFactor
# Drop rows with NaN values
df = df.dropna()

data_columns = [
    'rs_e_inairtemp_pc1', 'rs_e_inairtemp_pc2',
    'rs_e_oilpress_pc1', 'rs_e_oilpress_pc2',
    'rs_e_rpm_pc1', 'rs_e_rpm_pc2',
    'rs_e_wattemp_pc1', 'rs_e_wattemp_pc2',
    'rs_t_oiltemp_pc1', 'rs_t_oiltemp_pc2'
]



# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
df_normalized_lof = df.copy()
df_normalized_lof[data_columns] = scaler.fit_transform(df[data_columns])

df_normalized_iso_forest = df.copy()
df_normalized_iso_forest[data_columns] = scaler.fit_transform(df[data_columns])


LOCAL OUTLIER DETECTION

In [ ]:
# Now apply Local Outlier Factor for proximity-based outlier detection
# LocalOutlierFactor is one way to do proximity-based outlier detection
lof = LocalOutlierFactor(n_neighbors=20, algorithm='auto', metric='euclidean', contamination='auto')

# Fit the model and predict outliers
# The negative_outlier_factor_ is the opposite of the outlier score. Lower means more abnormal.
df_normalized_lof['outlier_score'] = lof.fit_predict(df_normalized_lof[data_columns])

# You can then filter the outliers based on the outlier score
# Outliers will have an outlier score of -1
outliers = df_normalized_lof[df_normalized_lof['outlier_score'] == -1]

# You can also examine the negative outlier factor to see how "outlying" a point is
df_normalized_lof['negative_outlier_factor'] = lof.negative_outlier_factor_

# Display outliers
print(outliers)

Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest
iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
df_normalized_iso_forest['iso_forest_score'] = iso_forest.fit_predict(df_normalized_iso_forest[data_columns])
